In [ ]:
# # Some pip install stuff that helped me, you probably won't need the boto3 install if you are running on AWS
# !pip install pyproj
# !pip install cartopy
# !pip uninstall shapely --y
# !pip install shapely --no-binary shapely
# !pip install boto3

In [ ]:
import os
import requests                            # Read from S3
from datetime import datetime, timedelta   # Datetime handling
from time import time                      # Timing

import boto3                               # AWS S3 access
import matplotlib.pyplot as plt            # Plotting library
import numpy as np                         # Scientific computing with Python
import cartopy, cartopy.crs as ccrs        # Plot maps
from netCDF4 import Dataset                # Read / Write NetCDF4 files
from pyproj import Proj                    # Cartographic projections and coordinate transformations library

goes = 16
bucket = "noaa-goes{}".format(goes) # change if we switch to a different satellite
prefix_base = "ABI-L2-CMIPF" # change if we use a different imaging instrument
aws_access_key_id = "" # fill in with your access id and key
aws_secret_access_key = ""
client = boto3.client('s3', 
                      aws_access_key_id=aws_access_key_id,
                      aws_secret_access_key=aws_secret_access_key)
# for converting to lat lon
default_lats_file = "g{}_lats_8km.txt".format(goes)
default_lons_file = "g{}_lons_8km.txt".format(goes)

def get_image(lat, lon, timestamp, 
              lats_file=default_lats_file, lons_file=default_lons_file,
              ref_grid_resolution_km=8, eyes=True):
  t0 = time()
  url = get_s3_url_for_timestamp(timestamp)
  url_time = time() - t0
  print("Acquired URL in {:.3f} seconds\n{}".format(url_time, url))
  save_and_plot_image(url, lat, lon, 
                      lats_file, lons_file, ref_grid_resolution_km, eyes)


def get_s3_url_for_timestamp(timestamp):
  date, time = timestamp.split("T")
  year = date.split("-")[0]
  days = (datetime.strptime(date, "%Y-%m-%d") - datetime.strptime(year, "%Y"))
  days = int(days.days + 1)
  hour, minute, _ = time.split(":")
  rounded_down_minute = minute[0] + "0"

  addendum = "{year}/{days}/{hour}/OR_ABI-L2-CMIPF-M6C02_G{goes}_s{year}{days}{hour}{minute}".format(
      year=year, days=days, hour=hour, minute=rounded_down_minute, goes=goes
  )

  prefix = "{}/{}".format(prefix_base, addendum)
  result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter='/')
  keys = [i["Key"] for i in result.get("Contents", [])]

  base_url = "https://noaa-goes{}.s3.amazonaws.com".format(goes)
  image = keys[0]
  url = "{}/{}".format(base_url, image)

  return url

def save_and_plot_image(url, lat, lon, 
                        lats_file, lons_file, ref_grid_resolution_km, eyes):
  t0 = time()
  # Open the GOES-R image
  image = url.split("/")[-1]
  resp = requests.get(url)
  image_file = Dataset(image, memory = resp.content)
  image_file_time = time()
  print("Acquired Image File in {:.3f} seconds".format(image_file_time - t0))
  
  # Get the image resolution
  band_resolution_km = getattr(image_file, 'spatial_resolution')
  band_resolution_km = float(band_resolution_km[:band_resolution_km.find("km")])
  
  min_lon, max_lon, min_lat, max_lat = lon - 1, lon + 1, lat - 1, lat + 1
  extent = [min_lon, min_lat, max_lon, max_lat]
  
  # Read the GOES-R lat lons as arrays (image_files created previously)
  # NOTE: load outside of for loop to make faster
  if not os.path.exists(lats_file):
    if lats_file == default_lats_file:
      create_default_lats_lons_files()
    else:
      raise Exception("Could not find lats lons files\n{}\n{}".format(lats_file, lons_file))
  lats = np.loadtxt(lats_file)
  lons = np.loadtxt(lons_file)
  
  # Calculate the lat lon pairs indexes for the desired extent
  idx_pair_1 = abs(lats-extent[1])+abs(lons-extent[0])
  max_lat_idx,min_lon_idx = np.unravel_index(idx_pair_1.argmin(),idx_pair_1.shape)
  idx_pair_2 = abs(lats-extent[3])+abs(lons-extent[2])
  min_lat_idx,max_lon_idx = np.unravel_index(idx_pair_2.argmin(),idx_pair_2.shape)
  
  # Adapt the reference indexes for the current image_file resolution
  min_lat_idx = min_lat_idx * int(ref_grid_resolution_km/band_resolution_km)
  min_lon_idx = min_lon_idx * int(ref_grid_resolution_km/band_resolution_km)
  max_lat_idx = max_lat_idx * int(ref_grid_resolution_km/band_resolution_km)
  max_lon_idx = max_lon_idx * int(ref_grid_resolution_km/band_resolution_km)
  
  # The projection x and y coordinates equals the scanning angle (in radians) multiplied by the satellite height
  sat_h = image_file.variables['goes_imager_projection'].perspective_point_height
  x = image_file.variables['x'][min_lon_idx:max_lon_idx] * sat_h
  y = image_file.variables['y'][min_lat_idx:max_lat_idx] * sat_h
  
  # Get the pixel values
  data = image_file.variables['CMI'][min_lat_idx:max_lat_idx,min_lon_idx:max_lon_idx][::1,::1]
  data_preprocessing_time = time()
  print("Preprocessed Data in {:.3f} seconds".format(data_preprocessing_time - image_file_time))
  
  # Choose the plot size (width x height, in inches)
  plt.figure(figsize=(7,7))

  # Satellite longitude
  sat_lon = image_file.variables['goes_imager_projection'].longitude_of_projection_origin
  # Satellite sweep
  sat_sweep = image_file.variables['goes_imager_projection'].sweep_angle_axis
  
  # Use the Geostationary projection in cartopy
  ax = plt.axes(projection=ccrs.Geostationary(central_longitude=sat_lon, satellite_height=sat_h))
  img_extent = (x.min(), x.max(), y.min(), y.max())
  
  # # Add a shapeimage_file
  # shapeimage_file = list(shpreader.Reader('ne_10m_admin_1_states_provinces.shp').geometries())
  # ax.add_geometries(shapeimage_file, ccrs.PlateCarree(), edgecolor='gold',facecolor='none', linewidth=0.3)
  
  if eyes:
    # Add coastlines, borders and gridlines
    ax.coastlines(resolution='10m', color='white', linewidth=0.8)
    #ax.add_feature(cartopy.feature.BORDERS, edgecolor='gray', linewidth=1.0)
    ax.gridlines(color='white', alpha=0.5, linestyle='--', linewidth=0.5)
    
    # Inserting Labels
    target_lats = [lat]
    target_lons = [lon]
    labels = ["Target"]
    x_offsets = [0.1]
    y_offsets = [0]
    
    ax.plot(target_lons, target_lats, 'bo', color='cyan', markersize=5, transform=ccrs.Geodetic())
    for label, xpt, ypt, x_offset, y_offset in zip(labels, target_lons, target_lats, x_offsets, y_offsets):
      ax.text(xpt+x_offset , ypt+y_offset, label, fontsize=12, fontweight='bold', zorder=8, color='gold', transform=ccrs.Geodetic())
  
  # Plot the image
  img = ax.imshow(data, vmin=0.0, vmax=0.7, extent=img_extent, origin='upper', cmap='gray')
  
  # Getting the image_file date
  add_seconds = int(image_file.variables['time_bounds'][0])
  date = datetime(2000,1,1,12) + timedelta(seconds=add_seconds)
  date = date.strftime('%d %B %Y %H:%M UTC')
  
  # Add a title
  plt.title('GOES-{} Band 02 (500m)'.format(goes), fontweight='bold', fontsize=10, loc='left')
  plt.title('Sub Region \n' + date, fontsize=10, loc='right')
  
  # Save the image
  plt.savefig('{}__{}_{}.png'.format(image[:-3], lat, lon))
  
  # Show the image
  plt.show()

  plot_time = time()
  print("Plotted in {:.3f} seconds".format(plot_time - data_preprocessing_time))

def create_default_lats_lons_files():
  # Reference file
  image = "OR_ABI-L2-CMIPF-M6C13_G{}_s20192061430442_e20192061440161_c20192061440233.nc".format(goes)
  image_url = "https://noaa-goes{}.s3.amazonaws.com/ABI-L2-CMIPF/2019/206/14/OR_ABI-L2-CMIPF-M6C13_G{}_s20192061430442_e20192061440161_c20192061440233.nc".format(goes, goes)
  resp = requests.get(image_url)

  image_file = Dataset(image, memory = resp.content)

  # Satellite height
  sat_h = image_file.variables['goes_imager_projection'].perspective_point_height
  # Satellite longitude
  sat_lon = image_file.variables['goes_imager_projection'].longitude_of_projection_origin
  # Satellite sweep
  sat_sweep = image_file.variables['goes_imager_projection'].sweep_angle_axis
  # The projection x and y coordinates equals
  # the scanning angle (in radians) multiplied by the satellite height (http://proj4.org/projections/geos.html)
  X = image_file.variables['x'][:][::4] * sat_h
  Y = image_file.variables['y'][:][::4] * sat_h
  # map object with pyproj
  # p = Proj(proj='geos', h=sat_h, lon_0=sat_lon, sweep=sat_sweep, a=6378137.0)
  p = Proj(proj='geos', h=sat_h, lon_0=sat_lon, sweep=sat_sweep)
  # Convert map points to latitude and longitude with the magic provided by Pyproj
  XX, YY = np.meshgrid(X, Y)
  lons, lats = p(XX, YY, inverse=True)
  
  # Pixels outside the globe as -9999
  mask = (lons == lons[0][0])
  lons[mask] = -9999
  lats[mask] = -9999
  
  np.savetxt(default_lats_file, lats, fmt='%.2f')
  np.savetxt(default_lons_file, lons, fmt='%.2f')

In [ ]:
# Berkeley
lat = 37.8719
lon = -122.2585
# New York
# lat = 40.7128
# lon = -74.0060
timestamp = "2019-07-31T18:05:05.0Z"